# UFC PREDICTION MODEL 1.0 (XGBoost) ###

Notes:
- To start enviroment: venv\Scripts\Activate.ps1

### Import required libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib # for model extraction

### Import Data

In [2]:
df = pd.read_csv("../Data\large_dataset.csv")

# Reverse the dataset to begin with the oldest fight
df = df.iloc[::-1].reset_index(drop=True)


<>:1: SyntaxWarning: invalid escape sequence '\l'
<>:1: SyntaxWarning: invalid escape sequence '\l'
C:\Users\fancy\AppData\Local\Temp\ipykernel_27040\3763960240.py:1: SyntaxWarning: invalid escape sequence '\l'
  df = pd.read_csv("../Data\large_dataset.csv")


### Setup Variables

In [3]:
# Create binary winner label
df['winner_binary'] = df['winner'].map({'Red': 1, 'Blue': 0})

diff_features = [
    'SLpM_total_diff', 'SApM_total_diff', 'sig_str_acc_total_diff',
    'td_acc_total_diff', 'str_def_total_diff', 'td_def_total_diff',
    'sub_avg_diff', 'td_avg_diff', 'age_diff', 'height_diff',
    'weight_diff', 'reach_diff', 'wins_total_diff', 'losses_total_diff'
]

# Drop rows with missing values in key columns
df = df.dropna(subset=diff_features + ['winner_binary'])

# Reverse for chronological ordering
df = df.iloc[::-1].reset_index(drop=True)

# Split dataset
split_idx = int(len(df) * (2 / 3))
train_df = df.iloc[:split_idx]
test_df = df.iloc[split_idx:]

# Define features and targets
X_train = train_df[diff_features]
y_train = train_df['winner_binary']
X_test = test_df[diff_features]
y_test = test_df['winner_binary']


# .55 Acc Version ----

# # Create binary winner label
# df['winner_binary'] = df['winner'].map({'Red': 1, 'Blue': 0})

# # List of pre-fight columns to generate diffs
# pre_fight_cols = [
#     'SLpM_total', 'SApM_total', 'sig_str_acc_total', 'td_acc_total',
#     'str_def_total', 'td_def_total', 'sub_avg', 'td_avg',
#     'age', 'height', 'weight', 'reach', 'wins_total', 'losses_total'
# ]

# # Compute raw diffs: Red - Blue
# for col in pre_fight_cols:
#     df[f"{col}_diff"] = df[f"r_{col}"] - df[f"b_{col}"]

# # Flip diffs for Blue wins so that all diffs are: winner - loser
# for col in pre_fight_cols:
#     diff_col = f"{col}_diff"
#     df.loc[df['winner_binary'] == 0, diff_col] = -df.loc[df['winner_binary'] == 0, diff_col]

# # 🚫 DO NOT overwrite the label
# # df['winner_binary'] = 1  ← REMOVE THIS LINE

# # Collect only the final diff columns for model input
# diff_features = [f"{col}_diff" for col in pre_fight_cols]

# # Drop rows with missing values in key columns
# df = df.dropna(subset=diff_features + ['winner_binary'])

# # Split data: 2/3 for training, 1/3 for testing
# split_idx = int(len(df) * (2 / 3))
# train_df = df.iloc[:split_idx]
# test_df = df.iloc[split_idx:]

# # Define features and targets
# X_train = train_df[diff_features]
# y_train = train_df['winner_binary']
# X_test = test_df[diff_features]
# y_test = test_df['winner_binary']

### Train the Model

In [4]:
# Train model
model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_estimators=100,
    max_depth=4,
    verbosity=0
)
model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

### Predict and Evaluate

In [5]:
# Predict + evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Display results
print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.642421398404505

Classification Report:
               precision    recall  f1-score   support

           0       0.41      0.57      0.48       617
           1       0.79      0.67      0.73      1514

    accuracy                           0.64      2131
   macro avg       0.60      0.62      0.60      2131
weighted avg       0.68      0.64      0.66      2131

Confusion Matrix:
 [[ 350  267]
 [ 495 1019]]


In [6]:
# Save model for external use
joblib.dump(model, "xgb_ufc_model.pkl")


['xgb_ufc_model.pkl']